## Tutorial of Interventions on Non-transformer Model: MLPs

In [1]:
__author__ = "Zhengxuan Wu"
__version__ = "12/20/2023"

### Overview

This tutorials show how to use this library on non-transformer models, such as MLPs. The set-ups are pretty much the same as standard transformer-based models.

### Set-up

In [2]:
try:
    # This library is our indicator that the required installs
    # need to be done.
    import transformers
    import sys
    sys.path.append("align-transformers/")
except ModuleNotFoundError:
    !git clone https://github.com/frankaging/align-transformers.git
    !pip install -r align-transformers/requirements.txt
    import sys
    sys.path.append("align-transformers/")

In [3]:
import sys
sys.path.append("../..")

import torch
import pandas as pd
from models.basic_utils import embed_to_distrib, top_vals, format_token
from models.configuration_alignable_model import AlignableRepresentationConfig, AlignableConfig
from models.alignable_base import AlignableModel
from models.interventions import VanillaIntervention, RotatedSpaceIntervention, LowRankRotatedSpaceIntervention
from models.mlp.modelings_mlp import MLPConfig
from models.mlp.modelings_alignable_mlp import create_mlp_classifier

%config InlineBackend.figure_formats = ['svg']
from plotnine import ggplot, geom_tile, aes, facet_wrap, theme, element_text, \
                     geom_bar, geom_hline, scale_y_log10

config, tokenizer, mlp =create_mlp_classifier(MLPConfig(h_dim=32))

[2023-12-21 18:30:32,696] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
loaded model


### Intervene in middle layer by partitioning representations into subspaces

MLP layer may contain only a single "token" representation each layer. As a result, we often want to intervene on a subspace of this "token" representation to localize a concept.

In [4]:
alignable_config = AlignableConfig(
    alignable_model_type=type(mlp),
    alignable_representations=[
        AlignableRepresentationConfig(
            0,
            "block_output",
            "pos",                              # mlp layer creates a single token reprs
            1,
            subspace_partition=[[0,16],[16,32]] # partition into two sets of subspaces
        ),
    ],
    alignable_interventions_type=RotatedSpaceIntervention,
)
alignable = AlignableModel(alignable_config, mlp)

base = {"inputs_embeds": torch.rand(1, 1, 32)}
source = {"inputs_embeds": torch.rand(1, 1, 32)}
print("base", alignable(base))
print("source", alignable(source))

base ((tensor([[[-0.1057,  0.0308]]]),), None)
source ((tensor([[[-0.0972,  0.0317]]]),), None)


In [5]:
_, counterfactual_outputs = alignable(
    base,
    [source],
    {"sources->base": ([[[0]]], [[[0]]])},
    subspaces = [[[0, 1]]]
)

In [6]:
counterfactual_outputs # this should be the same as source.

(tensor([[[-0.0972,  0.0317]]], grad_fn=<UnsafeViewBackward0>),)

### Intervene the subspace with multiple sources

In [9]:
alignable_config = AlignableConfig(
    alignable_model_type=type(mlp),
    alignable_representations=[
        AlignableRepresentationConfig(
            0,
            "block_output",
            "pos",                               # mlp layer creates a single token reprs
            1,
            alignable_low_rank_dimension=32,
            subspace_partition=[[0,16],[16,32]], # partition into two sets of subspaces
            intervention_link_key=0              # linked ones target the same subspace
        ),
        AlignableRepresentationConfig(
            0,
            "block_output",
            "pos",                               # mlp layer creates a single token reprs
            1,
            alignable_low_rank_dimension=32,
            subspace_partition=[[0,16],[16,32]], # partition into two sets of subspaces
            intervention_link_key=0              # linked ones target the same subspace
        ),
    ],
    alignable_interventions_type=LowRankRotatedSpaceIntervention,
)
alignable = AlignableModel(alignable_config, mlp)

base = {"inputs_embeds": torch.rand(10, 1, 32)}
source = {"inputs_embeds": torch.rand(10, 1, 32)}
print("base", alignable(base))
print("source", alignable(source))

base ((tensor([[[-0.1084,  0.0357]],

        [[-0.1035,  0.0311]],

        [[-0.0987,  0.0375]],

        [[-0.1008,  0.0359]],

        [[-0.0966,  0.0349]],

        [[-0.1000,  0.0316]],

        [[-0.1009,  0.0369]],

        [[-0.0981,  0.0340]],

        [[-0.1023,  0.0339]],

        [[-0.1036,  0.0320]]]),), None)
source ((tensor([[[-0.1035,  0.0372]],

        [[-0.1092,  0.0357]],

        [[-0.1013,  0.0417]],

        [[-0.1026,  0.0401]],

        [[-0.1024,  0.0322]],

        [[-0.1115,  0.0399]],

        [[-0.1092,  0.0360]],

        [[-0.0967,  0.0335]],

        [[-0.0968,  0.0293]],

        [[-0.0980,  0.0338]]]),), None)


In [10]:
_, counterfactual_outputs = alignable(
    base,
    [source, source],
    {"sources->base": ([[[0]]*10, [[0]]*10], [[[0]]*10, [[0]]*10])},
    subspaces = [[[0]]*10, [[1]]*10]
)
print(counterfactual_outputs) # this should be the same as the source output
counterfactual_outputs[0].sum().backward() # fake call to make sure gradient can be populated

(tensor([[[-0.1035,  0.0372]],

        [[-0.1092,  0.0357]],

        [[-0.1013,  0.0417]],

        [[-0.1026,  0.0401]],

        [[-0.1024,  0.0322]],

        [[-0.1115,  0.0399]],

        [[-0.1092,  0.0360]],

        [[-0.0967,  0.0335]],

        [[-0.0968,  0.0293]],

        [[-0.0980,  0.0338]]], grad_fn=<UnsafeViewBackward0>),)
